Item 21 Know How Closures Interact with Variable Scope 

Things to Remember
- Closures functions can refer to variables from any of the scopes in which they were defined
- By default, closures can't affect enclosing scopes by assigning variables
- Use the nonlocal statement to indicate when a closure can modify a variable in its enclosing scopes
- Avoid using nonlocal statements for anything beyond simple functions

background
- You want to sort a list of numbers but prioritize one group of numbers to come first.
- This pattern is useful when you are rendering a user interface and want important messages or exceptional events to be displayed before everything else 

In [ ]:
def sort_priority(values, group):
    def helper(x):
        # - helper can see the group argument and its content is {2, 3, 5, 7}
        # - helper can see the values argument (we don't need it in this example),
        #   whihch is an empty list, and that confuses me
        if x in group: 
            return (0, x)
        return (1, x)
    # - Python’s late binding behavior which 
    #   says that the values of variables used
    #   in closures are looked up at the time 
    #   the inner function is called
    # - my guess is that when the helper is called by 
    #   the sort the 'values' argument has been 
    #   re-initialized to an empty list but
    #   gets re-populated at later time 
    values.sort(key=helper)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
# 7 will appear before 1,4, and 6 as it's in the priority group
print(numbers) 

- Python supports closures. This is why the helper function is able to access the group argument for sort_priority.
- Functions are first class objects in Python, which means you can refer to them directly, assign them to variables, pass them as arguments to other functions, compare them in expressions and if statements. values.sort(key=helper) is an example of accepting a function as the key argument.
- Recap on how Python comparing sequences: it first compares items at the fist index, moving on to the next index if they are equal, and so on.   

In [ ]:
# now you want to return more info including if higher-priority items were seen at all
def sort_priority2(values, group):
    found = False # introduce a new variable to capture more info
    def helper(x):
        if x in group:
             # - we assume we are accessing the found variable
             #   defined in the sort_priority2 scope
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority2(numbers, group)
print('Found:', found) # It's false which is not what we expect
print(numbers) 

The order Python traverses the scope to resolve the reference of variables
- the current function's scope
- any enclosing scopes (such as other containing functions) 
- the scope of the module that contains the code (global scope)
- the built-in scope (than contains functions like len and str) 

In [ ]:
# scoping bug
# 
def sort_priority2(values, group):
    found = False # scope: sort_priority2
    def helper(x):
        if x in group:
            # - found has not been defined in the current scope (helper)
            # - Python treats the assignment as a variable definition
            # - so, you are not accessing the found variable defined in the
            #   sort_priority2 scope. Instead you are defining a new variable
            found = True # scope: helper
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found


solution I 
- use special syntax, nonlocal, to get data out of a closure
- nonlocal is used to tell Python it should start scope traversal to find the variables
- the traversal won't reach the module-level scope (to avoid polluting globals)
- use nonlocal only in simple functions as it will become hard to understand in long functions where the nonlocal statements and assignments to associated variables are far apart

In [22]:
def sort_priority3(values, group):
    found = False # scope: sort_priority2
    def helper(x):
        nonlocal found # 
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority3(numbers, group)
assert found == True

Solution II

- Wrap your state in a helper class

In [25]:
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    def __call__(self, x): 
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group) # sorter is a instance
numbers.sort(key=sorter) # can be used like a function
assert sorter.found is True

- The __call__ method enables Python programmers to write classes where the instances behave like functions and can be called like a function.
- so in this case sorter is an instance of Sorter can be treated as a function and assigned to the key parameter 